In [1]:
import pandas as pd
import streamlit as st
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from normal import do_normal
from sklearn.preprocessing import LabelEncoder, StandardScaler

changes made on branch main


In [ ]:
class DeepNN(nn.Module):
    def __init__(self, input_dim):
        super(DeepNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
def load_model():
    
    saved_model = DeepNN(input_dim)
    saved_model.load_state_dict(torch.load("lung_cancer_model.pth"))
    saved_model.eval()

    saved_model_wb = saved_model.model
    return saved_model_wb

# for idx, layer in enumerate(saved_model.model):
#     if isinstance(layer, nn.Linear):
#         print(f"Layer {idx} - Linear")
#         print(f"  Weights shape: {layer.weight.shape}")
#         print(f"  Weights:\n{layer.weight.data}\n")
#         print(f"  Biases shape: {layer.bias.shape}")
#         print(f"  Biases:\n{layer.bias.data}\n")

In [ ]:
# input_values = [ 0.9526,  1.2598, -1.1353, -1.1504, -0.9968, -1.0032,  0.9903, -1.4351,
#          0.8925, -1.1205,  0.8925,  0.8522,  0.7487,  1.0635,  0.8925]
# input_tensor = torch.tensor(input_values)

# print(input_tensor.shape)


torch.Size([15])


In [ ]:
# enc_vec = torch.tensor(np.random.uniform(0.9, 1.1, len(input_values)), dtype=torch.float32)
# print(enc_vec)

tensor([1.0644, 0.9512, 1.0228, 1.0692, 0.9853, 0.9568, 0.9209, 0.9285, 0.9774,
        1.0885, 1.0384, 1.0883, 0.9562, 0.9786, 1.0115])


In [ ]:
# an_plus_1 = torch.tensor(np.random.uniform(1, 2), dtype=torch.float32)
# print(an_plus_1)

tensor(1.6032)


In [ ]:
# enc_inp = enc_vec * input_tensor
# print(enc_inp)

tensor([ 1.0140,  1.1983, -1.1612, -1.2300, -0.9822, -0.9598,  0.9120, -1.3324,
         0.8723, -1.2196,  0.9267,  0.9274,  0.7159,  1.0408,  0.9028])


In [ ]:
# # Decryption vector
# dec_vec = 1 / enc_vec
# dec_vec = torch.cat((dec_vec, torch.tensor([1 / an_plus_1])))

In [ ]:
# k = 1 #temporary

In [ ]:
# eval_vec = dec_vec/k
# eval_vec = eval_vec.view(len(input_values)+1,1)

In [ ]:
# input_dim = len(enc_vec)
# output_dim = 64 

NameError: name 'enc_vec' is not defined

In [4]:
print(saved_model_wb)


NameError: name 'saved_model_wb' is not defined

In [12]:
print(saved_model_wb[0].bias.data.shape)
print(saved_model_wb[0].weight.data.shape)

torch.Size([64])
torch.Size([64, 15])


LAYER 1


In [13]:
activation_matrix = []

In [14]:
input_layer_weights = saved_model_wb[0].weight.data.T
print(input_layer_weights.shape)
input_layer_bias = saved_model_wb[0].bias.data
print(input_layer_bias.shape)

torch.Size([15, 64])
torch.Size([64])


In [5]:

def input_layer_calc(activation_matrix,saved_model_wb,enc_inp,eval_vec):
  input_layer_weights = saved_model_wb[0].weight.data.T
  input_layer_bias = saved_model_wb[0].bias.data
  for col_idx in range(input_layer_weights.shape[1]):
    # Element-wise multiplication with the column vector of input_layer_weights
    col_vector = input_layer_weights[:, col_idx]
    elementwise_product = enc_inp * col_vector
    
    # Add (an+1 * bias for that column)
    elementwise_product = torch.cat((elementwise_product, (an_plus_1 * input_layer_bias[col_idx]).unsqueeze(0)))
    
    # Store as a row vector in activation matrix
    activation_matrix.append(elementwise_product)

# Convert activation_matrix to a tensor
    activation_matrix = torch.stack(activation_matrix)
    print(activation_matrix)
    print(activation_matrix.shape)
    relu_output = []

    for row_vector in activation_matrix:
        # Compute the sum of elements in the row vector
        row_sum = torch.matmul(row_vector,eval_vec)
        # row_sum = row_vector.sum()
        # Apply ReLU activation
        relu_value = F.relu(row_sum)
        if relu_value == 0:
            row_vector.zero_()
        # Append the result to relu_output
        relu_output.append(relu_value)

    relu_output = torch.tensor(relu_output)
    activation_matrix = torch.stack([row_vector if relu_value > 0 else torch.zeros_like(row_vector) 
                                                for row_vector, relu_value in zip(activation_matrix, relu_output)])
    return activation_matrix


    

In [16]:

# for col_idx in range(input_layer_weights.shape[1]):
#     # Element-wise multiplication with the column vector of input_layer_weights
#     col_vector = input_layer_weights[:, col_idx]
#     elementwise_product = enc_inp * col_vector
    
#     # Add (an+1 * bias for that column)
#     elementwise_product = torch.cat((elementwise_product, (an_plus_1 * input_layer_bias[col_idx]).unsqueeze(0)))
    
#     # Store as a row vector in activation matrix
#     activation_matrix.append(elementwise_product)

# # Convert activation_matrix to a tensor
# activation_matrix = torch.stack(activation_matrix)
# print(activation_matrix)
# print(activation_matrix.shape)

In [17]:
# relu_output = []

# for row_vector in activation_matrix:
#     # Compute the sum of elements in the row vector
#     row_sum = torch.matmul(row_vector,eval_vec)
#     # row_sum = row_vector.sum()
#     # Apply ReLU activation
#     relu_value = F.relu(row_sum)
#     if relu_value == 0:
#         row_vector.zero_()
#     # Append the result to relu_output
#     relu_output.append(relu_value)

# relu_output = torch.tensor(relu_output)

In [18]:
# activation_matrix = torch.stack([row_vector if relu_value > 0 else torch.zeros_like(row_vector) 
#                                         for row_vector, relu_value in zip(activation_matrix, relu_output)])

In [6]:
def intermediate_layer_calc(activation_matrix,saved_model_wb,eval_vec):
  for i in range(2,12,2):
    intermediate_weights = saved_model_wb[i].weight.data.T
    intermediate_biases= saved_model_wb[i].bias.data
    
    activation_matrix_new = []

    for col_idx in range(intermediate_weights.shape[1]):
    # Get the column vector of intermediate_weights
      col_vector = intermediate_weights[:, col_idx]
    
    # Scale each row of activation_matrix using the corresponding weight
      scaled_activation = activation_matrix * col_vector.view(-1,1)
    
    # Sum the scaled activation matrix column-wise to generate a row vector
      row_vector = scaled_activation.sum(dim=0)
    
    # Add (an+1 * bias for the current column) to the last element of the row vector
      row_vector[-1] += an_plus_1 * intermediate_biases[col_idx]
    
    # Append the row vector to activation_matrix_new
      activation_matrix_new.append(row_vector)

    activation_matrix_new = torch.stack(activation_matrix_new)

    relu_output = []

    for row_vector in activation_matrix_new:
    # Compute the sum of elements in the row vector
      row_sum = torch.matmul(row_vector,eval_vec)
    # Apply ReLU activation
      relu_value = F.relu(row_sum)
      if relu_value == 0:
        row_vector.zero_()
    # Append the result to relu_output
      relu_output.append(relu_value)

    relu_output = torch.tensor(relu_output)
    activation_matrix = torch.stack([row_vector if relu_value > 0 else torch.zeros_like(row_vector) 
                                        for row_vector, relu_value in zip(activation_matrix_new, relu_output)])
    print(activation_matrix.shape)
    
  return activation_matrix

In [20]:
activation_matrix = input_layer_calc(activation_matrix,input_layer_weights,input_layer_bias)

tensor([[-0.0733, -0.0914, -0.2212, -0.2995, -0.1976,  0.0763, -0.2186,  0.2842,
         -0.0270, -0.1663,  0.1792,  0.1800,  0.0154, -0.1407, -0.1496,  0.1440]])
torch.Size([1, 16])


In [21]:

activation_matrix = intermediate_layer_calc(activation_matrix)

torch.Size([48, 16])
torch.Size([32, 16])
torch.Size([16, 16])
torch.Size([8, 16])
torch.Size([1, 16])


In [22]:
print(activation_matrix.shape)

dec_vec = dec_vec.view(16,1)
print(dec_vec.shape)

torch.Size([1, 16])
torch.Size([16, 1])


In [23]:
decrypted_output = torch.matmul(activation_matrix,dec_vec)

In [24]:
print(decrypted_output)

tensor([[0.2952]])


In [25]:
final_ans = torch.sigmoid(decrypted_output[0][0])

print(final_ans)

tensor(0.5733)


In [7]:
test_data= pd.read_csv("test.csv")
test_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1.049781,-0.204116,-1.135292,0.869300,1.003241,0.996769,-1.009756,-1.435063,0.892475,0.892475,-1.120480,0.852207,-1.335584,1.063501,-1.120480
1,0.952579,-1.180082,0.880830,0.869300,1.003241,0.996769,0.990338,-1.435063,0.892475,-1.120480,0.892475,-1.173424,-1.335584,1.063501,0.892475
2,0.952579,1.259833,-1.135292,-1.150351,-0.996769,-1.003241,0.990338,-1.435063,0.892475,-1.120480,0.892475,0.852207,0.748736,1.063501,0.892475
3,0.952579,0.893846,-1.135292,0.869300,-0.996769,0.996769,0.990338,0.696833,0.892475,0.892475,0.892475,0.852207,-1.335584,1.063501,0.892475
4,0.952579,0.283867,-1.135292,0.869300,1.003241,-1.003241,-1.009756,0.696833,-1.120480,0.892475,0.892475,0.852207,0.748736,1.063501,0.892475


In [8]:
y_test = pd.read_csv("y_test.csv")
y_test.head() 

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [14]:
input_dim=15

In [11]:

enc_vec = torch.tensor(np.random.uniform(0.9, 1.1, 15), dtype=torch.float32)
an_plus_1 = torch.tensor(np.random.uniform(1, 2), dtype=torch.float32)
# Decryption vector
dec_vec = 1 / enc_vec
dec_vec = torch.cat((dec_vec, torch.tensor([1 / an_plus_1])))
k = 1 #temporary
eval_vec = dec_vec/k
eval_vec = eval_vec.view(16,1)

In [12]:
def encrypt_data(input_values):
  input_tensor = torch.tensor(input_values)
  enc_inp = enc_inp = enc_vec * input_tensor
  return enc_inp
  

In [15]:
saved_model_wb = load_model()

In [16]:
def decrypt_ans(activation_matrix,dec_vec):
  dec_vec = dec_vec.view(16,1)
  decrypted_output = torch.matmul(activation_matrix,dec_vec)
  final_ans = torch.sigmoid(decrypted_output[0][0])
  return final_ans

In [17]:
def bsavnn_model_pred(input_values,eval_vec):
  enc_inp = encrypt_data(input_values)
  activation_matrix =[]
  activation_matrix = input_layer_calc(activation_matrix,saved_model_wb,enc_inp,eval_vec)
  activation_matrix = intermediate_layer_calc(activation_matrix,saved_model_wb,eval_vec)
  return activation_matrix

  


In [26]:
%%time
y_pred = []
for index, row in test_data.iterrows():
  row_input = encrypt_data(row) 
  row_input = encrypt_data(row).float()
  eval_vec = eval_vec.float()

  final_activation_matrix = bsavnn_model_pred(row_input,eval_vec)
  final_ans = decrypt_ans(final_activation_matrix,dec_vec)
  predicted_classes = (final_ans > 0.5).float()
  y_pred.append(predicted_classes.item())

from sklearn.metrics import accuracy_score, classification_report
acc = accuracy_score(y_test, y_pred)
print("\n✅ Test Accuracy:", round(acc * 100, 2), "%")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

  

C:\Users\susha\AppData\Local\Temp\ipykernel_17400\4274733193.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  input_tensor = torch.tensor(input_values)
C:\Users\susha\AppData\Local\Temp\ipykernel_17400\4274733193.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(input_values)


tensor([[ 0.0049,  0.0008, -0.0541,  0.0033, -0.0024, -0.0420, -0.0362,  0.0055,
          0.0060,  0.0134, -0.0054,  0.0056, -0.0019, -0.0140, -0.0104, -0.0020]])
torch.Size([1, 16])
torch.Size([48, 16])
torch.Size([32, 16])
torch.Size([16, 16])
torch.Size([8, 16])
torch.Size([1, 16])
tensor([[-0.0045,  0.0046,  0.0420,  0.0033, -0.0024, -0.0420,  0.0355,  0.0055,
          0.0060, -0.0168,  0.0043, -0.0077, -0.0019, -0.0140,  0.0083, -0.0020]])
torch.Size([1, 16])
torch.Size([48, 16])
torch.Size([32, 16])
torch.Size([16, 16])
torch.Size([8, 16])
torch.Size([1, 16])
tensor([[-0.0045, -0.0049, -0.0541, -0.0043,  0.0023,  0.0423,  0.0355,  0.0055,
          0.0060, -0.0168,  0.0043,  0.0056,  0.0011, -0.0140,  0.0083, -0.0020]])
torch.Size([1, 16])
torch.Size([48, 16])
torch.Size([32, 16])
torch.Size([16, 16])
torch.Size([8, 16])
torch.Size([1, 16])
tensor([[-0.0045, -0.0035, -0.0541,  0.0033,  0.0023, -0.0420,  0.0355, -0.0027,
          0.0060,  0.0134,  0.0043,  0.0056, -0.0019, -0.0

c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,


✅ Test Accuracy: 96.77 %

📊 Classification Report:
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         2
         1.0       0.97      1.00      0.98        60

    accuracy                           0.97        62
   macro avg       0.48      0.50      0.49        62
weighted avg       0.94      0.97      0.95        62



c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\susha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,